In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Load File
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

In [2]:
#Find total unique players
purchase_data_copy = purchase_data
total_players = len(purchase_data_copy["SN"].unique())
print("Total Players: ", total_players)

Total Players:  576


## Purchasing Analysis (Total)

In [3]:
# Finding the number of unique items
unique_items = purchase_data_copy["Item ID"].nunique()
print("Unique Items: ", unique_items)

Unique Items:  183


In [4]:
# Finding the average price 
average_price = purchase_data_copy["Price"].mean()
print("Average Price: ", average_price)

Average Price:  3.050987179487176


In [5]:
# Finding total purchases
total_purchases = purchase_data_copy["Purchase ID"].count()
print("Total Purchases: ",total_purchases)

Total Purchases:  780


In [6]:
# Finding total revenue
total_revenue = purchase_data_copy["Price"].sum()
total_revenue

2379.77

In [7]:
# Display the summary data frame and round to two decimal places
purchase_analysis_totals = pd.DataFrame({
    "Unique Items": [unique_items],
    "Average Price":[average_price],
    "Total Purchases":[total_purchases],
    "Total Revenue": [total_revenue]
})
purchase_analysis_totals.round(2)

,Unique Items,Average Price,Total Purchases,Total Revenue
0,183,3.05,780,2379.77


## Gender Demographics

In [8]:
# Clean data so there are no duplicates
demographic_df = purchase_data.loc[:,["Gender", "SN", "Age"]]
demographic_df = demographic_df.drop_duplicates()

In [9]:
# Create data frame of the different genders
gender_demographics = pd.DataFrame(demographic_df["Gender"].value_counts())

In [10]:
percentage_of_players = gender_demographics["Gender"]/total_players * 100

In [11]:
# Converting the count into percentages and adding a column to the DataFrame
percentage_of_players = gender_demographics["Gender"]/total_players * 100
gender_demographics["Percentage of Players"] = percentage_of_players

In [12]:
# Format for percentages
gender_demographics["Percentage of Players"] = gender_demographics["Percentage of Players"].map("{0:,.2f}%".format)

In [13]:
#Display gender demographics overview
gender_demographics.head()

,Gender,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

In [14]:
# Set up groups, return values
purchase_count = purchase_data.groupby(["Gender"]).count()["Price"]
avg_purchase_price = purchase_data.groupby(["Gender"]).mean()["Price"]
purchase_value = purchase_data.groupby(["Gender"]).sum()["Price"]
avg_purchase_total_pp = purchase_value/gender_demographics["Gender"]

In [15]:
# Create data frame to hold values
purchase_analysis = pd.DataFrame({
    "Purchase Count": purchase_count,
    "Average Purchase Price": avg_purchase_price,
    "Total Purchase Value": purchase_value,
    "Avg Total Purchase per Person": avg_purchase_total_pp
})

In [16]:
# Format for currency
purchase_analysis["Average Purchase Price"] = purchase_analysis["Average Purchase Price"].map("${0:,.2f}".format)
purchase_analysis["Total Purchase Value"] = purchase_analysis["Total Purchase Value"].map("${0:,.2f}".format)
purchase_analysis["Avg Total Purchase per Person"] = purchase_analysis["Avg Total Purchase per Person"].map("${0:,.2f}".format)

purchase_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [17]:
# Clean data so there are no duplicates
age_df = purchase_data.loc[:,["Age", "SN", "Price"]]
age_df = age_df.drop_duplicates("SN")

In [18]:
# Create bins for age ranges
age_bins = [0,9,14,19,24,29,34,39,100]

In [19]:
# Create group names
group_labels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

In [20]:
# Create output and new column to df 
age_df["Age Groups"] = pd.cut(age_df['Age'],age_bins,labels = group_labels)

In [21]:
# Set up group for Age Group Bins
age_group_df = age_df.groupby("Age Groups").count()

In [22]:
# Convert the count into percentages and add a column to the Data Frame
age_df_format = age_group_df
percent_players_age_count = (age_df_format["Age"]/total_players) * 100
age_df_format["Percentage of Players"] = percent_players_age_count

In [23]:
# Format for percentages
age_df_format["Percentage of Players"] = age_df_format["Percentage of Players"].map("{0:,.2f}%".format)

In [24]:
# Rename age column to total count and call columns needed
age_df_format = age_df_format.rename(columns={'Age': 'Total Count'})
age_group_df_final = age_df_format[["Total Count", "Percentage of Players"]]
age_group_df_final

,Total Count,Percentage of Players
Age Groups,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [25]:
# Bin purchase_data frame by age
purchase_data["Age Ranges"] = pd.cut(purchase_data['Age'],age_bins,labels = group_labels)

In [26]:
# Set up groups, return values
purchase_count_age = purchase_data.groupby(["Age Ranges"]).count()["Price"]
purchase_count_age_combined = purchase_data.groupby(["Age Ranges", "SN"]).nunique()["Price"]
purchase_count_age_unique = purchase_count_age_combined.groupby(["Age Ranges"]).count()
avg_purchase_price_age = purchase_data.groupby(["Age Ranges"]).mean()["Price"]
purchase_value_age = purchase_data.groupby(["Age Ranges"]).sum()["Price"]
avg_purchase_total_pp_age = purchase_value_age/purchase_count_age_unique

In [27]:
# Create data frame to hold values
purchase_age_analysis = pd.DataFrame({
    "Purchase Count": purchase_count_age,
    "Average Purchase Price": avg_purchase_price_age,
    "Total Purchase Value": purchase_value_age,
    "Avg Total Purchase per Person": avg_purchase_total_pp_age
})

In [28]:
# Format for currency
purchase_age_analysis["Average Purchase Price"] = purchase_age_analysis["Average Purchase Price"].map("${0:,.2f}".format)
purchase_age_analysis["Total Purchase Value"] = purchase_age_analysis["Total Purchase Value"].map("${0:,.2f}".format)
purchase_age_analysis["Avg Total Purchase per Person"] = purchase_age_analysis["Avg Total Purchase per Person"].map("${0:,.2f}".format)

In [29]:
# Display purchase age range analysis
purchase_age_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

In [30]:
# Create smaller data frame
top_spender = purchase_data.loc[:,["SN", "Price", "Item Name"]]

In [31]:
# Set up groups, return values
purchase_count_top_spend = top_spender.groupby(["SN"]).count()["Item Name"]
avg_purchase_price_top_spend = top_spender.groupby(["SN"]).mean()["Price"]
purchase_value_top_spend = top_spender.groupby(["SN"]).sum()["Price"]

In [32]:
# Create summary df to hold the values
top_spenders_analysis = pd.DataFrame({
    "Purchase Count": purchase_count_top_spend,
    "Average Purchase Price": avg_purchase_price_top_spend,
    "Total Purchase Value": purchase_value_top_spend,
})

In [33]:
# Sort by Total Purchase Value to get top spenders
top_spenders_analysis = top_spenders_analysis.sort_values(by = "Total Purchase Value", ascending = False)

In [34]:
# Format for currency
top_spenders_analysis["Average Purchase Price"] = top_spenders_analysis["Average Purchase Price"].map("${0:,.2f}".format)
top_spenders_analysis["Total Purchase Value"] = top_spenders_analysis["Total Purchase Value"].map("${0:,.2f}".format)

In [35]:
#Display top 5 spenders 
top_spenders_analysis.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [36]:
# Create smaller data frame
most_popular = purchase_data.loc[:,["Item ID", "Item Name", "Price"]]

In [37]:
# Set up groups, return values
purchase_count_most_popular = most_popular.groupby(["Item ID"]).count()["Item Name"]
purchase_value_most_popular = most_popular.groupby(["Item ID"]).sum()["Price"]

In [38]:
# Create summary df to hold the values
most_popular_analysis = pd.DataFrame({
    "Purchase Count": purchase_count_most_popular,
    "Total Purchase Value": purchase_value_most_popular,
})

In [39]:
# Join two tables by merging
merge_most_popular = pd.merge(most_popular.dropna(), most_popular_analysis.dropna(), on ="Item ID", how="left")

In [40]:
# Remove duplicates
merge_most_popular_dedup = merge_most_popular.drop_duplicates("Item ID", keep="first")

In [41]:
# Sort by Purchase Count and Item ID to find most popular items
merge_most_popular_dedup = merge_most_popular_dedup.sort_values(by = ["Purchase Count", "Item ID"], ascending = False)

In [42]:
# Rename to keep integrity of last version of df for future use
most_popular_analysis = merge_most_popular_dedup

In [43]:
# Format for currency
most_popular_analysis["Price"] = most_popular_analysis["Purchase Count"].map("${0:,.2f}".format)
most_popular_analysis["Purchase Count"] = most_popular_analysis["Purchase Count"].map("${0:,.2f}".format)
most_popular_analysis["Total Purchase Value"] = most_popular_analysis["Total Purchase Value"].map("${0:,.2f}".format)

In [44]:
# Set index and display top 5 most popular items 
most_popular_analysis.set_index(["Item ID", "Item Name"]).head()

,,Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$12.00,$12.00,$50.76
145,Fiery Glass Crusader,$9.00,$9.00,$41.22
108,"Extraction, Quickblade Of Trembling Hands",$9.00,$9.00,$31.77
82,Nirvana,$9.00,$9.00,$44.10
103,Singed Scalpel,$8.00,$8.00,$34.80


## Most Profitable Items

In [60]:
# Create smaller data frame
most_profitable = purchase_data.loc[:,["Item ID", "Item Name", "Price"]]

In [61]:
# Set up groups, return values
purchase_count_most_profitable = most_profitable.groupby(["Item ID"]).count()["Item Name"]
purchase_value_most_profitable = most_profitable.groupby(["Item ID"]).sum()["Price"]

In [62]:
# Create summary df to hold the values
most_profitable_analysis = pd.DataFrame({
    "Purchase Count": purchase_count_most_profitable,
    "Total Purchase Value": purchase_value_most_profitable,
})

In [63]:
# Join two tables by merging
merge_most_profitable = pd.merge(most_profitable.dropna(), most_profitable_analysis.dropna(), on ="Item ID", how="left")

In [64]:
# Remove duplicates
merge_most_profitable_dedup = merge_most_profitable.drop_duplicates("Item ID", keep="first")

In [65]:
# Sort data by Total Purchase Value to obtain most profitable
most_profitable_analysis = merge_most_profitable_dedup.sort_values(by = "Total Purchase Value", ascending = False)

In [66]:
# Format for currency
most_profitable_analysis["Price"] = most_profitable_analysis["Purchase Count"].map("${0:,.2f}".format)
most_profitable_analysis["Purchase Count"] = most_profitable_analysis["Purchase Count"].map("${0:,.2f}".format)
most_profitable_analysis["Total Purchase Value"] = most_profitable_analysis["Total Purchase Value"].map("${0:,.2f}".format)

In [67]:
# Set index and display top 5 most 
most_profitable_analysis.set_index(["Item ID", "Item Name"]).head()

,,Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$12.00,$12.00,$50.76
82,Nirvana,$9.00,$9.00,$44.10
145,Fiery Glass Crusader,$9.00,$9.00,$41.22
92,Final Critic,$8.00,$8.00,$39.04
103,Singed Scalpel,$8.00,$8.00,$34.80


## Key Observations 

In [47]:
#Item 178 (Oathbreaker) is both the most profitable and most popular suggesting there is opportunity to increase profitable growth.

In [48]:
#Items 82 (Nirvana) & 145 (Fiery Glass Crusader) also show promise to be profitable growth drivers second to item 178. 

In [49]:
#Target age range looks to be 20-25 with a heavy male skew and sweet spot for pricing among top 5 spenders is ~$3.80 to $4.40 per item.